In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame

In [2]:
csv_file_beers = "Resources/beers.csv"
beers_df = pd.read_csv(csv_file_beers)
beers_df.head()

,Unnamed: 0,abv,ibu,id,name,style,brewery_id,ounces
0,0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0


In [3]:
newbeer_df = beers_df[['abv', 'id', 'name','style', 'brewery_id', 'ounces']].copy()

newbeer_df.head()

,abv,id,name,style,brewery_id,ounces
0,0.050,1436,Pub Beer,American Pale Lager,408,12.0
1,0.066,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,0.071,2264,Rise of the Phoenix,American IPA,177,12.0
3,0.090,2263,Sinister,American Double / Imperial IPA,177,12.0
4,0.075,2262,Sex and Candy,American IPA,177,12.0


In [4]:
csv_file_breweries = "Resources/breweries.csv"
breweries_df = pd.read_csv(csv_file_breweries)
breweries_df.head()

,Unnamed: 0,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


### Create new data with select columns

In [5]:
newbrew_df = breweries_df[['Unnamed: 0','name', 'city', 'state']].copy()
newbrew_df = newbrew_df.rename(columns={'name':'brewery_name','Unnamed: 0':'brewery_id'})
newbrew_df.set_index('brewery_id')

,brewery_name,city,state
brewery_id,,,
0,NorthGate Brewing,Minneapolis,MN
1,Against the Grain Brewery,Louisville,KY
2,Jack's Abby Craft Lagers,Framingham,MA
3,Mike Hess Brewing Company,San Diego,CA
4,Fort Point Beer Company,San Francisco,CA
...,...,...,...
553,Covington Brewhouse,Covington,LA
554,Dave's Brewfarm,Wilson,WI
555,Ukiah Brewing Company,Ukiah,CA


### Connect to local database

In [6]:
rds_connection_string = "postgres:postgres@localhost:5432/Project_2"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [7]:
engine.table_names()

['breweries', 'beers', 'merged', 'results']

In [8]:
newbrew_df.to_csv(r"Resources/breweries2.csv")
newbeer_df.to_csv(r"Resources/beers2.csv")

### Use pandas to load csv converted DataFrame into database

In [ ]:
newbrew_df.to_sql(name='breweries', con=engine, if_exists='append', index=False)

In [ ]:
newbeer_df.to_sql(name='beers', con=engine, if_exists='append', index=True)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [9]:
pd.read_sql_query('select * from beers', con=engine).head()

,index,abv,id,name,style,brewery_id,ounces
0,0,0.050,1436,Pub Beer,American Pale Lager,408,12
1,1,0.066,2265,Devil's Cup,American Pale Ale (APA),177,12
2,2,0.071,2264,Rise of the Phoenix,American IPA,177,12
3,3,0.090,2263,Sinister,American Double / Imperial IPA,177,12
4,4,0.075,2262,Sex and Candy,American IPA,177,12


### Confirm data has been added by querying the customer_location table

In [10]:
pd.read_sql_query('select * from breweries', con=engine).head()

,brewery_id,brewery_name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


In [11]:
merged_df = pd.merge(newbrew_df,newbeer_df,on='brewery_id')
merged_df

,brewery_id,brewery_name,city,state,abv,id,name,style,ounces
0,0,NorthGate Brewing,Minneapolis,MN,0.045,2692,Get Together,American IPA,16.0
1,0,NorthGate Brewing,Minneapolis,MN,0.049,2691,Maggie's Leap,Milk / Sweet Stout,16.0
2,0,NorthGate Brewing,Minneapolis,MN,0.048,2690,Wall's End,English Brown Ale,16.0
3,0,NorthGate Brewing,Minneapolis,MN,0.060,2689,Pumpion,Pumpkin Ale,16.0
4,0,NorthGate Brewing,Minneapolis,MN,0.060,2688,Stronghold,American Porter,16.0
...,...,...,...,...,...,...,...,...,...
2405,556,Butternuts Beer and Ale,Garrattsville,NY,0.049,52,Heinnieweisse Weissebier,Hefeweizen,12.0
2406,556,Butternuts Beer and Ale,Garrattsville,NY,0.068,51,Snapperhead IPA,American IPA,12.0
2407,556,Butternuts Beer and Ale,Garrattsville,NY,0.049,50,Moo Thunder Stout,Milk / Sweet Stout,12.0
2408,556,Butternuts Beer and Ale,Garrattsville,NY,0.043,49,Porkslap Pale Ale,American Pale Ale (APA),12.0


In [ ]:
merged_df.to_sql(name='merged', con=engine, if_exists='append', index=True)